In [27]:
from flask import Flask
app = Flask(__name__)
@app.route('/')
def hello_world():
    return 'Hello world'

In [2]:
import datetime as dt
import numpy as np
import pandas as pd
from app import hello_world

ModuleNotFoundError: No module named 'sqlalchemy'

In [46]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [36]:
#from flask import Flask, jsonify
engine = create_engine("sqlite:///hawaii.sqlite")
engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()


None


In [ ]:
Base.prepare(engine, reflect=True)
print(Base.prepare(engine, reflect=True))

In [44]:
Base.classes.values()


[]

In [42]:
Measurement = Base.classes.measurement
Station = Base.classes.station
session = Session(engine)


AttributeError: measurement

In [26]:
def welcome():
    return(
    '''
    Welcome to the Climate Analysis API!
    Available Routes:
    /api/v1.0/precipitation
    /api/v1.0/stations
    /api/v1.0/tobs
    /api/v1.0/temp/start/end
    ''')

In [ ]:
@app.route("/api/v1.0/precipitation")

In [ ]:
def precipitation():
   prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
   precipitation = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= prev_year).all()
    precip = {date: prcp for date, prcp in precipitation}
   return jsonify(precip)

In [ ]:
@app.route("/api/v1.0/stations")

In [ ]:
def stations():
    results = session.query(Station.station).all()
    stations = list(np.ravel(results))
    return jsonify(stations=stations)

In [ ]:
@app.route("/api/v1.0/tobs")

In [ ]:
def temp_monthly():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    results = session.query(Measurement.tobs).\
        filter(Measurement.station == 'USC00519281').\
        filter(Measurement.date >= prev_year).all()
    temps = list(np.ravel(results))
    return jsonify(temps=temps)

In [ ]:
def stats(start=None, end=None):
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]

    if not end:
        results = session.query(*sel).\
            filter(Measurement.date >= start).all()
        temps = list(np.ravel(results))
        return jsonify(temps)

    results = session.query(*sel).\
        filter(Measurement.date >= start).\
        filter(Measurement.date <= end).all()
    temps = list(np.ravel(results))
    return jsonify(temps)